# Date Input Widget

Use Workflows [parameters](https://docs.descarteslabs.com/descarteslabs/workflows/docs/types/identifier.html#descarteslabs.workflows.types.identifier.parameter) to interactively pick the start and end dates for a max-NDVI composite.

You can run the following cells using `Shift-Enter`.

In [ ]:
# keep logging quiet
import logging; logging.getLogger().setLevel(logging.INFO); logging.captureWarnings(True)
import datetime

import ipyleaflet
import ipywidgets

import descarteslabs.workflows as wf

In [ ]:
s2 = wf.ImageCollection.from_id(
    "sentinel-2:L1C",
    start_datetime=wf.parameter("begin", wf.Datetime),
    end_datetime=wf.parameter("end", wf.Datetime),
)

# Filter clouds
s2 = s2.filter(lambda img: img.properties["cloud_fraction_0"] < 0.1)

# Compute NDVI
red, nir = s2.unpack_bands("red nir")
ndvi = (nir - red) / (nir + red)

# Make NDVI max composite
max_ndvi = ndvi.max(axis="images")

In [ ]:
lyr = max_ndvi.visualize(
    name="S2 NDVI",
    scales=[(0, 1)],
    colormap="viridis",
    begin=ipywidgets.DatePicker(value=datetime.date(2019, 1, 1)),
    end=ipywidgets.DatePicker(value=datetime.date(2019, 2, 1)),
)

In [ ]:
control = ipyleaflet.WidgetControl(widget=lyr.parameters.widget, position='bottomleft')
wf.map.add_control(control)

In [ ]:
wf.map